# Data Cleaning

## **Import package**

In [2]:
import pandas as pd
import numpy as np
import re
# from symspellpy import SymSpell, Verbosity
from collections import Counter
# from underthesea import word_tokenize
import json
from google.colab import drive
from difflib import get_close_matches


In [16]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Read data**

In [4]:
df = pd.read_json(
    "/content/drive/MyDrive/statlearning/ner_labeled_data_full.jsonl",
    lines=True,
    encoding="utf-8"
)

In [5]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 36,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 291...",https://vietstock.vn/2025/05/thi-truong-chung-...


In [6]:
stock_list = []
with open("/content/drive/MyDrive/statlearning/all_vn_stock_codes.txt", "r", encoding="utf-8") as f:
    stock_list = [line.strip().upper() for line in f if line.strip()]

In [7]:
def build_common_company_names(df, top_n=500):
    company_names = []
    for entities in df["entities"]:
        for ent in entities:
            if ent["type"] == "COMPANY":
                company_names.append(ent["text"].strip())
    most_common = Counter(company_names).most_common(top_n)
    return [name for name, _ in most_common]

In [8]:
company_list = build_common_company_names(df)

In [54]:
company_list

['Fed',
 'UBCKNN',
 'NHNN',
 'CTCK',
 'KBSV',
 'VNDIRECT',
 'Dragon Capital',
 'ACBS',
 'Aseansc',
 'VCBS',
 'VietinBank',
 'Bộ Tài chính',
 'BSC',
 'DNSE',
 'MSCI',
 'Vingroup',
 'Yuanta',
 'KIS',
 'Vietcap',
 'VinaCapital',
 'Vietcombank',
 'FTSE Russell',
 'FTSE',
 'SCIC',
 'VDSC',
 'SSV',
 'VSDC',
 'Vinamilk',
 'PYN Elite',
 'Masan',
 'Vietstock',
 'Novaland',
 'PAN',
 'BIDV',
 'UOB',
 'Masan Consumer',
 'Pomina',
 'Ủy ban Chứng khoán Nhà nước',
 'Berkshire Hathaway',
 'SSIR',
 'HAGL',
 'SJC',
 'IFC',
 'TPS',
 'DAS',
 'VNG',
 'Apple',
 'Coteccons',
 'FPTS',
 'Agriseco Research',
 'Ngân hàng Nhà nước',
 'F88',
 'SK Group',
 'Novagroup',
 'Vinhomes',
 'CTCK BIDV',
 'Sacombank',
 'CTCK KB Việt Nam',
 'HDI Global SE',
 'FLC',
 'SSIAM',
 'VietstockFinance',
 'HSC',
 'BETA',
 'Vinatex',
 'Mirae Asset',
 'Beta',
 'Hòa Phát',
 'CTCK Asean',
 'SSI Research',
 'Vinpearl',
 'YSVN',
 'Berkshire',
 'BlackRock',
 'EVN',
 'FDI',
 'Hải An',
 'SSI',
 'CTCK Tiên Phong',
 'Nam Long',
 'CTCP',
 'Asean

In [9]:
def safe_get_span(text, value):
    try:
        start = text.index(value)
        end = start + len(value)
        return start, end
    except ValueError:
        return -1, -1


In [10]:
def clean_entities_and_text(text, entities, stock_list, company_list, cutoff=0.8):
    new_text = text
    temp_entities = []

    offset = 0  # Để tính sự thay đổi độ dài sau khi thay thế từ

    for ent in entities:
        ent_text = ent["text"]
        ent_type = ent["type"]
        ent_start = ent.get("start")
        ent_end = ent.get("end")

        if ent_start is None or ent_end is None:
            ent_start, ent_end = safe_get_span(new_text, ent_text)
            if ent_start == -1:
                temp_entities.append(ent)
                continue  # Không tìm thấy thì bỏ qua xử lý sửa

        # Lấy đoạn gốc trong text hiện tại
        current_text = new_text[ent_start + offset: ent_end + offset]

        if ent_type == "STOCK":
            corrected = ent_text.upper()
            if corrected in stock_list:
                # Thay đúng vị trí trong text
                new_text = new_text[:ent_start + offset] + corrected + new_text[ent_end + offset:]
                delta = len(corrected) - (ent_end - ent_start)
                temp_entities.append({
                    "text": corrected,
                    "type": ent_type,
                    "start": ent_start + offset,
                    "end": ent_start + offset + len(corrected)
                })
                offset += delta
            else:
                # Giữ nguyên nhưng vẫn gán lại vị trí
                temp_entities.append({
                    "text": ent_text,
                    "type": ent_type,
                    "start": ent_start + offset,
                    "end": ent_end + offset
                })

        elif ent_type == "COMPANY":
            matched = get_close_matches(ent_text, company_list, n=1, cutoff=cutoff)
            corrected = matched[0] if matched else ent_text
            new_text = new_text[:ent_start + offset] + corrected + new_text[ent_end + offset:]
            delta = len(corrected) - (ent_end - ent_start)
            temp_entities.append({
                "text": corrected,
                "type": ent_type,
                "start": ent_start + offset,
                "end": ent_start + offset + len(corrected)
            })
            offset += delta

        else:
            # Giữ nguyên hoàn toàn cho các loại khác
            temp_entities.append({
                "text": ent_text,
                "type": ent_type,
                "start": ent_start + offset,
                "end": ent_end + offset
            })

    return new_text, temp_entities


In [11]:
for i in range(len(df)):
    raw_text = df.at[i, "text"]
    raw_ents = df.at[i, "entities"]

    new_text, new_ents = clean_entities_and_text(raw_text, raw_ents, stock_list, company_list)

    # Ghi đè vào chính cột gốc
    df.at[i, "text"] = new_text
    df.at[i, "entities"] = new_ents


In [14]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệHAGp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 36,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 291...",https://vietstock.vn/2025/05/thi-truong-chung-...


In [21]:
# import json

with open("/content/drive/MyDrive/statlearning/cleaned_data_full.jsonl", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "text": row["text"],
            "entities": row["entities"],
            "source" : row['source']
        }, f, ensure_ascii=False)
        f.write("\n")


OSError: [Errno 30] Read-only file system: '/content/drive/MyDrive/statlearning/cleaned_data_full.jsonl'